In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df.drop(['RowNumber','CustomerId','Surname','Geography'],axis=1,inplace=True)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,StandardScaler
le=LabelEncoder()
df['Gender']=le.fit_transform(df['Gender'])

In [6]:
dict(zip(le.classes_,  le.transform(le.classes_) ))

{'Female': 0, 'Male': 1}

In [8]:
from sklearn.model_selection import train_test_split
x=df.drop(['EstimatedSalary'],axis=1)
y=df['EstimatedSalary']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [9]:
scalar=StandardScaler()

In [10]:

x_train=scalar.fit_transform(x_train)
x_test=scalar.transform(x_test)

In [12]:

x_train=pd.DataFrame(x_train,columns=scalar.get_feature_names_out())
x_test=pd.DataFrame(x_test,columns=scalar.get_feature_names_out())
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((8000, 9), (2000, 9), (8000,), (2000,))

In [13]:
x_train.columns

Index(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'Exited'],
      dtype='object')

In [14]:
# ann model for regression

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [33]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(x_train.shape[1] ,)),
    Dense(32,activation='relu'),
    Dense(1)
])

C:\Users\PS001028870\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [34]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                      │ (None, 64)                  │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,753 (10.75 KB)

 Trainable params: 2,753 (10.75 KB)

 Non-trainable params: 0 (0.00 B)

In [35]:
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

In [36]:
# set up tensorboard
import os
import datetime
logdir = os.path.join("logsreg/fit/", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tf_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [37]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [46]:
history=model.fit(x_train,y_train,validation_data=(x_test,y_test),batch_size=64 ,epochs=100,callbacks=[tf_callback,early_stopping_callback])

Epoch 1/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 49603.6289 - mae: 49603.6289 - val_loss: 50167.0273 - val_mae: 50167.0273
Epoch 2/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 50145.9648 - mae: 50145.9648 - val_loss: 50159.6875 - val_mae: 50159.6875
Epoch 3/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 49636.5117 - mae: 49636.5117 - val_loss: 50163.5469 - val_mae: 50163.5469
Epoch 4/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 49556.1523 - mae: 49556.1523 - val_loss: 50166.8711 - val_mae: 50166.8711
Epoch 5/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 49273.1602 - mae: 49273.1602 - val_loss: 50158.9688 - val_mae: 50158.9688
Epoch 6/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 49529.7305 - mae: 49529.7305 - val_loss: 50158.7227 - val_mae: 50158.7227
Epoch 7/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 49774.9219 - mae: 49774.9219 - val_loss: 50162.2461 - val_mae: 50162.2461
Epoch 8/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - 

In [39]:
#evaluate

In [40]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [42]:
%tensorboard --log_dir logsreg/fit


ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
Error: A logdir or db must be specified. For example `tensorboard --logdir mylogdir` or `tensorboard --db sqlite:~/.tensorboard.db`. Run `tensorboard --helpfull` for details and examples.

In [44]:
x_train.shape


(8000, 9)

In [45]:
8000/250

32.0